# CurricularFace
CurricularFace was modified from arcface to achive the good result and rubost than arcface. 
you can read paper [CurricularFace](https://arxiv.org/pdf/2004.00288.pdf)

My code include: 

**Algorithm**: arcface, CurricularFace

**Augmentation**: Mixup,Cutmix

**Another technique** SAM optimizer, adamp,..

**training stratify**: small size (224) to big size ( 512)

**learning finder**: on-going


In [ ]:
!pip install adamp
!git clone https://github.com/tks1998/pytorch-lr-finder.git
%cd pytorch-lr-finder 
!pip install .

In [ ]:
cd ..

In [ ]:
# ==================================================== Library ==================================================== #
# sys
import sys
import os
import time
import random
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from tqdm import tqdm
import glob

# dataloader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
from torchvision.datasets.folder import *
from torch.nn.parameter import Parameter
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

import cv2
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

### import algorithm
import math
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

### visulization
import tensorflow as tf
from matplotlib import pyplot as plt
torch.manual_seed(1234)
### timm module
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master/')
import timm
from adamp import AdamP
from torch_lr_finder import LRFinder
from torch.cuda import amp
from torch.autograd import Variable
from torch.cuda.amp import autocast, GradScaler

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(1213)

In [ ]:
class CFG:
    # path
    
    path_info = '../input/happywhale-cropped-dataset-yolov5-ds/train.csv'
    root_data_2022 = '../input/happywhale-cropped-dataset-yolov5-ds/train_images/'
    
    # dataloader
    input_size = [224]
    batch_size = 128
    worker = 4
    phase_idx = 0

    # model
    use_pretrained = True
    model_name = "eca_nfnet_l0"
    num_classes = 15587
    embedding_size = 512
    adaptive = False
    dropout = 0.5
    
    ### metric
    metric = 'CurricularFace' # 'arcface', 'cosface', 'adacos','adaptive_arcface'
    use_fc = True
    s = 30
    margin = 0.5
    
    ### losss 
    MIXUP = True
    CUTMIX = False
    ls_eps = 0.0
    theta_zero = 0.785
    MIXUP_PROB = 0.5
    MIXUP_ALPHA = 1.0
    ### learning rate finder 
    skip_start = 10
    skip_end = 5

    # optimizer
    SAM = False
    lr = 1e-4
    min_lr = 1e-6
    weight_decay = 3e-5
    scheduler = 'ReduceLROnPlateau'
    epochs = [120]
    T_max = 6
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    T_MULT = 2
    T_0 = 5
    gamma = 1.5
    
    # other parameter
    seed = 42
    print_freq = 100
    early_stop = 10
    train_continue = False
    freq_save = 10
    valid_every = 5
    loss_name = "HardMining"  # or focal loss
    use_amp = False
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    tensor_log = "./log/product_"      + model_name + "_" + metric + "_" + loss_name + "_" + scheduler
    path_save = "./out_model/product_" + model_name + "_" + metric + "_" + loss_name + "_" + scheduler +".pt"

    # make dir
    os.makedirs('log', exist_ok=True)
    os.makedirs('out_model', exist_ok=True)


In [ ]:
class min_edge_crop(A.ImageOnlyTransform):
    
    def __init__(self, p: float = 0.5, always_apply=True):
        super().__init__(always_apply, p)
        self.position = 'center'

    def apply(self, img, **params):
        assert self.position in ['center', 'left', 'right'], "position must either be: left, center or right"

        h, w = img.shape[:2]

        if h == w:
            return img

        min_edge = min(h, w)
        if h > min_edge:
            if self.position == "left":
                img = img[:min_edge]
            elif self.position == "center":
                d = (h - min_edge) // 2
                img = img[d:-d] if d != 0 else img

                if h % 2 != 0:
                    img = img[1:]
            else:
                img = img[-min_edge:]

        if w > min_edge:
            if self.position == "left":
                img = img[:, :min_edge]
            elif self.position == "center":
                d = (w - min_edge) // 2
                img = img[:, d:-d] if d != 0 else img

                if w % 2 != 0:
                    img = img[:, 1:]
            else:
                img = img[:, -min_edge:]
        return img
def get_augmentation(phase,input_size):
    if phase == "train":
        return  A.Compose([
#                     min_edge_crop(),
                    A.Resize(height=input_size, width=input_size),
                     A.ToGray(p=0.01),
                     A.OneOf([
                       A.GaussNoise(var_limit=[10, 50]),
                       A.GaussianBlur(),
                       A.MotionBlur(),
                       A.MedianBlur(),
                      ], p=0.2),
                    A.OneOf([
                       A.OpticalDistortion(distort_limit=1.0),
                       A.GridDistortion(num_steps=5, distort_limit=1.),
                       A.ElasticTransform(alpha=3),
                   ], p=0.2),
                     A.OneOf([
                         A.CLAHE(),
                         A.RandomBrightnessContrast(),
                     ], p=0.25),
                     A.HueSaturationValue(p=0.25),
                    A.ShiftScaleRotate(p=0.5, shift_limit=0.0625, scale_limit=0.2, rotate_limit=20),
#                     A.Cutout(max_h_size=int(input_size * 0.1), max_w_size=int(input_size * 0.1), num_holes=5, p=0.5),
                    A.Normalize(),
                    ToTensorV2()
                ])
    elif phase in ['test','valid']:
        return A.Compose([
#             min_edge_crop(),
            A.Resize(height=input_size, width=input_size),
            A.Normalize(),
            ToTensorV2()
        ])


In [ ]:
class whale_huback():
    
    def __init__(self, df, transform = None):
        
        self.df = df.reset_index()
        self.transform = transform
                
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self,index):
        
        img_path, class_id = self.df.loc[index, 'image_path'], self.df.loc[index,'individual_id']
        sample = cv2.imread(img_path)
        if sample is None:
            print(img_path)
            
        sample = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            sample = self.transform(image=sample)["image"]

        return sample, torch.tensor(class_id)

In [ ]:
info = pd.read_csv(CFG.path_info)
info['image'] = CFG.root_data_2022+info['image']
print(len(info['image']))

info = info.sample(frac=1).reset_index(drop=True)

mapping = {}
mapping_invert = {}
index=-1

all_class = info['individual_id'].unique()
np.random.shuffle(all_class)
for class_id in all_class:
    if class_id not in mapping.keys():
        index+=1
        mapping[class_id] = index
        mapping_invert[index] = class_id
info['individual_id'] = info['individual_id'].apply(lambda class_id: mapping[class_id]) 
balanced_data = pd.DataFrame([])
for class_id in info['individual_id'].unique():
    sub_df = info[info['individual_id'] == class_id]
    if sub_df.shape[0] < 2:
        sub_df_repeat = pd.concat([sub_df]*3)
        balanced_data = pd.concat([balanced_data,sub_df_repeat])
info = pd.concat([balanced_data,info])
train,valid = train_test_split(info, random_state=42, stratify=info['individual_id'], train_size=0.7)
train.to_csv("train_arcface.csv")
valid.to_csv("valid_arcface.csv")

In [ ]:
import torch

class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

In [ ]:
class whale_huback():
    
    def __init__(self, df, transform = None):
        
        self.df = df.reset_index()
        self.transform = transform
                
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self,index):
        
        img_path, class_id = self.df.loc[index, 'image_path'], self.df.loc[index,'individual_id']
        sample = cv2.imread(img_path)
        if sample is None:
            print(img_path)
            
        sample = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            sample = self.transform(image=sample)["image"]

        return sample, torch.tensor(class_id)
    


# In[7]:


import torch
from torch import nn
from tqdm import tqdm

class FocalLoss(nn.Module):

    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = torch.nn.CrossEntropyLoss()

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()
class HardMining(nn.Module):
    def __init__(self, save_rate = 2):
        super(HardMining, self).__init__()
        self.save_rate = save_rate
        self.ce = nn.CrossEntropyLoss()
    
    def forward(self, input, target):
        batch_size = input.shape[0]
        loss = self.ce(input, target)
        ind_sorted = torch.argsort(-loss) # from big to small
        num_saved = int(self.save_rate * batch_size)
        ind_update = ind_sorted[:num_saved]
        loss_final = torch.sum(self.ce(input[ind_update], target[ind_update]))
        return loss_final
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def train_fn(dataloader, model, criterion, optimizer, epoch, CFG):
    model.train()
    loss_score = AverageMeter()

    tk0 = tqdm(enumerate(dataloader), total=len(dataloader))
    for bi, d in tk0:
        batch_size = d[0].shape[0]

        images = d[0]
        targets = d[1]
        
        images = images.to(CFG.device)
        targets = targets.to(CFG.device)
        
        
        if CFG.MIXUP:
            images, label_a, label_b, lam = mixup_data(images, targets, CFG.device, CFG.MIXUP_PROB, CFG.MIXUP_ALPHA)
            images, label_a, label_b = map(Variable, (images, label_a, label_b))
        elif CFG.CUTMIX:
            images, label_a, label_b, lam = cutmix_data(images, targets, CFG.device, CFG.MIXUP_PROB, CFG.MIXUP_ALPHA)
            images, label_a, label_b, lam = map(Variable, (inputs, label_a, label_b))

        with amp.autocast(enabled=True):
            output = model(images, targets)
            if CFG.MIXUP or CFG.CUTMIX:
                loss = mixup_criterion(criterion, output, label_a, label_b, lam)
            else:
                loss = criterion(output, targets)

        optimizer.zero_grad()

        if CFG.adaptive:
            loss +=CFG.lambda_*torch.mean(model.final.m)
        
        loss.backward()
        optimizer.step()
        loss_score.update(loss.detach().item(), batch_size)
        tk0.set_postfix(Train_Loss=loss_score.avg, Epoch=epoch, LR=optimizer.param_groups[0]['lr'])
    return loss_score


def eval_fn(data_loader, model, criterion, CFG):
    loss_score = AverageMeter()

    model.eval()
    tk0 = tqdm(enumerate(data_loader), total=len(data_loader))

    with torch.no_grad():
        for bi, d in tk0:
            batch_size = d[0].size()[0]

            image = d[0]
            targets = d[1]

            image = image.to(CFG.device)
            targets = targets.to(CFG.device)

            output = model(image, targets)

            loss = criterion(output, targets)

            loss_score.update(loss.detach().item(), batch_size)
            tk0.set_postfix(Eval_Loss=loss_score.avg)

    return loss_score



In [ ]:
### model
def l2_norm(input, axis = 1):
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)

    return output

class CurricularFace(nn.Module):
    r"""Implement of CurricularFace (https://arxiv.org/pdf/2004.00288.pdf):
    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        device_id: the ID of GPU where the model will be trained by model parallel. 
                       if device_id=None, it will be trained on CPU without model parallel.
        m: margin
        s: scale of outputs
    """
    def __init__(self, in_features, out_features, m = 0.5, s = 64.):
        super(CurricularFace, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.m = torch.tensor(m).to(CFG.device)
        self.s = torch.tensor(s).to(CFG.device)
        self.cos_m = torch.cos(self.m)
        self.sin_m = torch.sin(self.m)
        torch.pi = torch.acos(torch.zeros(1))* 2 
        torch.pi = torch.pi.to(CFG.device)
        self.threshold = torch.cos(torch.pi - self.m)
        self.mm = torch.sin(torch.pi - self.m) * self.m
        self.kernel = Parameter(torch.Tensor(in_features, out_features))
        torch_t = torch.zeros(1).float() 
        self.register_buffer('t', torch_t.to(CFG.device))
        nn.init.normal_(self.kernel, std=0.01)
        
    @autocast()
    def forward(self, embbedings, label):
        embbedings = l2_norm(embbedings, axis = 1)
        embbedings = embbedings.to(CFG.device)
        kernel_norm = l2_norm(self.kernel, axis = 0)
        kernel_norm = kernel_norm.to(CFG.device)
        cos_theta = torch.mm(embbedings, kernel_norm)
        cos_theta = cos_theta.clamp(-1, 1)  # for numerical stability
        cos_theta = cos_theta.to(CFG.device)
        cos_theta = cos_theta.float()
        with torch.no_grad():
            origin_cos = cos_theta.clone()
        target_logit = cos_theta[torch.arange(0, embbedings.size(0)), label].view(-1, 1)

        sin_theta = torch.sqrt(1.0 - torch.pow(target_logit, 2))
        cos_theta_m = target_logit * self.cos_m - sin_theta * self.sin_m #cos(target+margin)
        mask = cos_theta > cos_theta_m
        
        final_target_logit = torch.where(target_logit > self.threshold, cos_theta_m, target_logit - self.mm)
        hard_example = cos_theta[mask]
#         hard_example = torch.tensor(hard_example)
        hard_example = hard_example.to(CFG.device)
        hard_example = hard_example.float()
        with torch.no_grad():
            self.t = target_logit.mean() * 0.01 + (1 - 0.01) * self.t
        cos_theta[mask] = hard_example * (self.t + hard_example)
        cos_theta.scatter_(1, label.view(-1, 1).long(), final_target_logit)
        output = cos_theta * self.s
        return output

class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = torch.tensor(s).to(CFG.device)
        self.m = torch.tensor(m).to(CFG.device)
        self.ls_eps = ls_eps  # label smoothing
        self.weight = Parameter(torch.FloatTensor(out_features, in_features)).to(CFG.device)
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = torch.cos(self.m).to(CFG.device)
        self.sin_m = torch.sin(self.m).to(CFG.device)
        torch.pi = torch.acos(torch.zeros(1))* 2 
        torch.pi = torch.pi.to(CFG.device)
        self.th = torch.cos(torch.pi - self.m)
        self.mm = torch.sin(torch.pi - self.m) * self.m
#         self.th = self.th.to(CFG.device)
#         self.mm = self.mm.to(CFG.device)
    @autocast()
    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        cosine = cosine.to(CFG.device)
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2)).to(CFG.device)
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=CFG.device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output
    
class Model(nn.Module):

    def __init__(self, n_classes, model_name='efficientnet_b3', use_fc=False, fc_dim=512,                 dropout=0.0, metric='softmax', s=30.0, margin=0.50, ls_eps=0.0,                 theta_zero=0.785, pretrained=False):
        """
        :param n_classes:
        :param model_name: name of model from pretrainedmodels
            e.g. resnet50, resnext101_32x4d, pnasnet5large
        :param pooling: One of ('SPoC', 'MAC', 'RMAC', 'GeM', 'Rpool', 'Flatten', 'CompactBilinearPooling')
        :param metric: One of ('arcface', 'cosface', 'softmax')
        """
        super(Model, self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=True)
#         self.backbone.load_state_dict(torch.load(CFG.orginal_pretrain_model, map_location=CFG.device))
#         if 'coat' in CFG.model_name:
#             final_in_features = self.backbone.head.in_features
#         else:
#             final_in_features = self.backbone.classifier.in_features
#             self.backbone.classifier = nn.Identity()
#             self.backbone.global_pool = nn.Identity()

        if 'efficientnet' in CFG.model_name:
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif 'nfnet' in CFG.model_name:
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()
            
        self.backbone.head = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc
        if use_fc:
            self.dropout = nn.Dropout(p=dropout)
            self.fc = nn.Linear(final_in_features, fc_dim)
            self.bn = nn.BatchNorm1d(fc_dim)
            self._init_params()
            final_in_features = fc_dim

        self.metric = metric
        if metric == 'arcface':
            self.final = ArcMarginProduct(final_in_features, n_classes,
                                          s=s, m=margin, easy_margin=False, ls_eps=ls_eps)
        elif metric == 'cosface':
            self.final = AddMarginProduct(final_in_features, n_classes, s=s, m=margin, device=CFG.device)
        elif metric == 'adacos':
            self.final = AdaCos(final_in_features, n_classes, m=margin, theta_zero=theta_zero)
        elif metric == 'adaptive_arcface':
            self.final = Arcface_adaptive_margin(final_in_features, n_classes, s=s, m=margin,                                                 device=CFG.device, easy_margin=False, ls_eps=ls_eps)
        else:
            self.final = nn.Linear(final_in_features, n_classes)

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, x, label):
        feature = self.extract_feat(x)
        if self.metric in ('arcface', 'cosface', 'adacos','adaptive_arcface'):
            logits = self.final(feature, label)
        else:
            logits = self.final(feature)
        return logits

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)

        return x



In [ ]:
model

In [ ]:

def get_scheduler(CFG,optimizer):
    if CFG.scheduler=='CosineAnnealingLR':
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
    elif CFG.scheduler=='CosineAnnealingWarmRestarts':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer=optimizer,
                T_0=len(dataloader['train']) * CFG.T_0 * CFG.T_MULT,
                T_mult=CFG.T_MULT,
                eta_min=CFG.min_lr,
        )
    elif CFG.scheduler== 'ReduceLROnPlateau':
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer=optimizer,
            min_lr=CFG.min_lr
        )
    return scheduler


# In[10]:


def get_model(CFG):
    model = Model(**{
                'n_classes':CFG.num_classes,
                'model_name':CFG.model_name,
                'use_fc':CFG.use_fc,
                'fc_dim':CFG.embedding_size,
                'dropout':CFG.dropout,
                'metric':CFG.metric,
                's':CFG.s,
                'margin':CFG.margin,
                'ls_eps':CFG.ls_eps,
                'theta_zero':CFG.theta_zero,
                'pretrained':False})
    return model


# In[11]:


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


# In[12]:


def mixup_data(x, y, gpu, mixup_prob=0.5, alpha=1.0):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if random.uniform(0, 1) < mixup_prob:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).cuda(gpu)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam



In [ ]:
for index, input_size in enumerate(CFG.input_size):
    print("training size {}",format(input_size))
    dataset = {
        phase: whale_huback(eval(phase),transform = get_augmentation(phase=phase,input_size=input_size)) \
        for phase in ['train','valid']
    }

    dataloader = { phase: Data.DataLoader(dataset=dataset[phase], num_workers=(len(CFG.input_size)-index)*CFG.worker, batch_size=CFG.batch_size,                                           shuffle=(phase=='train'),pin_memory = (phase=='train'), drop_last=True)                   for phase in ['train','valid'] }

    model = get_model(CFG)

    model.to(CFG.device)

    criterion = FocalLoss(gamma=1.5) if CFG.loss_name == "focal_loss" else nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)

    scheduler = get_scheduler(CFG,optimizer)
    
    best_loss = np.inf
    if CFG.train_continue==True or index == 1:
        print("continue train")
        path_previous_save = "./out_model/product_" + CFG.model_name + "_" + CFG.metric + "_" + CFG.loss_name + "_" + CFG.scheduler + str(CFG.input_size[index-1]) +".pt"
        checkpoint = torch.load(path_previous_save)
        model.load_state_dict(checkpoint['model_state_dict']) 
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        criterion = checkpoint['criterion']
        scheduler = checkpoint['scheduler']
        best_loss = checkpoint['best_loss']
    
    
    visualization = tf.summary.create_file_writer(CFG.tensor_log)

    with visualization.as_default():
        
        for epoch in range(0, CFG.epochs[index]+1):
            for phase in ['train','valid']:
                if phase == 'train':
                    loss = train_fn(dataloader['train'], model,criterion, optimizer, epoch=epoch,CFG=CFG)
                else:
                    loss = eval_fn(dataloader['valid'], model, criterion, CFG=CFG)
                tf.summary.scalar('./loss/{}/{}'.format(input_size,phase), loss.avg, step=epoch)
                print('epoch {} loss {}'.format(epoch,loss.avg)) 
                if phase == 'valid':
                    path_save = "./out_model/product_" + CFG.model_name + "_" + CFG.metric + "_" + CFG.loss_name + "_" + CFG.scheduler + str(input_size) +".pt"
                    if best_loss > loss.avg:
                        best_loss = loss.avg
                        torch.save({
                            'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'criterion': criterion,
                            'scheduler': scheduler,
                            'best_loss':best_loss
                        }, CFG.path_save)
                        not_improve=0
                    else:
                        not_improve+=1
            scheduler.step(loss.item())
            if not_improve == CFG.early_stop:
                print("early_stop at epoch ", epoch)
                break
    ### del  
    del model, dataset, dataloader, criterion, scheduler, loss

